In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException,TimeoutException, ElementClickInterceptedException
import undetected_chromedriver as uc
import json
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [7]:
link_produk = 'https://shopee.co.id/search?category=11043572&keyword=bibit%20kaliandra'
path = r"C:/Users/KBIBKU- BNBYTO/Documents/Scraping data looker/chromedriver-win64/chromedriver.exe"

In [8]:
options = uc.ChromeOptions()
options.add_argument("--start-maximized")

driver = uc.Chrome(options=options)

# Buka halaman Shopee
driver.set_window_size(1300,800)
driver.get(link_produk)
time.sleep(5)

# Tambahkan cookies
with open('cookies_.json', 'r') as f:
    cookies = json.load(f)

for cookie in cookies:
    cookie.pop("sameSite", None)
    cookie.pop("expiry", None)
    driver.add_cookie(cookie)

time.sleep(3)
driver.refresh()
time.sleep(5)

In [10]:
halaman = 1
product_data = []
duplicate_count = 0
processed_products = set()

while True:
    print(f"Scraping halaman ke-{halaman}")
    produk_list = driver.find_elements(By.XPATH, "//li[contains(@class, 'shopee-search-item-result__item')]")
    print(f"Jumlah elemen ditemukan: {len(produk_list)}")

    page_duplicate_count = 0
    
    for i, produk in enumerate(produk_list, start=1):
        print(f"Proses data ke-{i}")
        try:
            nama = produk.find_elements(By.XPATH, ".//div[contains(@class, 'line-clamp-2')]")
            gambar = produk.find_elements(By.XPATH, ".//img")
            harga = produk.find_elements(By.XPATH, ".//div[contains(@class, 'truncate flex items-baseline')]")
            rating = produk.find_elements(By.XPATH, ".//div[contains(@class, 'flex-none flex items-center')]")
            terjual = produk.find_elements(By.XPATH, ".//div[contains(@class, 'truncate text-shopee-black87')]")
            lokasi = produk.find_elements(By.XPATH, ".//div[contains(@class, 'flex-shrink min-w-0 truncate text-shopee-black54 font-extralight text-sp10')]")
            link = produk.find_elements(By.XPATH, ".//a")

            # Ambil data jika ada, jika kosong beri nilai None
            nama_text = nama[0].text if nama else "None"
            gambar_src = gambar[0].get_attribute('src') if gambar else "None"
            harga_text = harga[0].text if harga else "None"
            rating_text = rating[0].text if rating else "0"
            terjual_text = terjual[0].text if terjual else "0"
            lokasi_text = lokasi[0].text if lokasi else "None"
            link_href = link[0].get_attribute('href') if link else "None"

            product_id = f"{nama_text}_{harga_text}"
            
            if product_id in processed_products:
                print(f"Produk '{nama_text}' sudah diproses sebelumnya.")
                page_duplicate_count += 1
                if page_duplicate_count >= 3:
                    print(f"Ditemukan {page_duplicate_count} produk duplikat. Sepertinya kita sudah di halaman terakhir.")
                    break
                continue
                
            processed_products.add(product_id)
            
            # Cetak hasil
            print(f"Nama: {nama_text}")
#             print(f"Gambar: {gambar_src}")
#             print(f"Harga: {harga_text}")
#             print(f"Rating: {rating_text}")
#             print(f"Terjual: {terjual_text}")
#             print(f"Lokasi: {lokasi_text}")
#             print(f"Link: {link_href}")
#             print('-------')
    
            product_data.append({
            "nama produk": nama_text,
            "gambar": gambar_src,
            "harga": harga_text,
            "rating": rating_text,
            "terjual": terjual_text,
            "lokasi": lokasi_text,
            "link": link_href
            })
        
        except Exception as e:
            print(f"Error memproses data ke-{i}: {e}")
    
    if page_duplicate_count >= 3:
        print("Ditemukan 3 atau lebih produk duplikat. Scraping selesai.")
        break
    
    try:
        tombol_next = driver.find_element(By.XPATH, "//a[contains(@class, 'shopee-icon-button shopee-icon-button--right')]")
        aria_disabled = tombol_next.get_attribute('aria-disabled')
        
        if aria_disabled == 'true':
            print("Tombol halaman selanjutnya tidak aktif. Scraping selesai.")
            break
            
        tombol_next.click()
        print(f"Berhasil klik tombol halaman selanjutnya.")
        halaman += 1
        time.sleep(3)
    except (NoSuchElementException, ElementClickInterceptedException):
        print("Tombol halaman selanjutnya tidak ditemukan atau tidak dapat diklik.")
        break

Scraping halaman ke-1
Jumlah elemen ditemukan: 5
Proses data ke-1
Nama: Benih Kaliandra Merah Pagoda Pakan Ternak isi 10gram /Bibit Pakan Ternak Super
Proses data ke-2
Nama: Bibit kaliandra merah/pagoda pakan ternak kambing/cabutan/jebolan ambil langsung setelah ada orderan grosir bibit kaliandra termurah
Proses data ke-3
Nama: Bibit Pakan Ternak Kaliandra Merah
Proses data ke-4
Nama: BIBIT KALIANDRA PAKAN TERNAK KAMBING DLL UKURAN 20-60CM
Proses data ke-5
Nama: Bibit Javanese Trigona/Levicep/Pot Madu Tipis/Trigona/Klanceng/Teuwul/Kele-kele/Linot/Sarang/Lebah/Setup
Berhasil klik tombol halaman selanjutnya.
Scraping halaman ke-2
Jumlah elemen ditemukan: 5
Proses data ke-1
Produk 'Benih Kaliandra Merah Pagoda Pakan Ternak isi 10gram /Bibit Pakan Ternak Super' sudah diproses sebelumnya.
Proses data ke-2
Produk 'Bibit kaliandra merah/pagoda pakan ternak kambing/cabutan/jebolan ambil langsung setelah ada orderan grosir bibit kaliandra termurah' sudah diproses sebelumnya.
Proses data ke-3
Pr

In [11]:
df = pd.DataFrame(product_data)

In [12]:
df

,nama produk,gambar,harga,rating,terjual,lokasi,link
0,Benih Kaliandra Merah Pagoda Pakan Ternak isi ...,https://down-id.img.susercontent.com/file/9c7e...,Rp\n8.500,4.8,227 sold,Kab. Wonogiri,https://shopee.co.id/Benih-Kaliandra-Merah-Pag...
1,Bibit kaliandra merah/pagoda pakan ternak kamb...,https://down-id.img.susercontent.com/file/516b...,Rp\n1.000,4.4,"1,2RB sold",Kab. Trenggalek,https://shopee.co.id/Bibit-kaliandra-merah-pag...
2,Bibit Pakan Ternak Kaliandra Merah,https://down-id.img.susercontent.com/file/3f69...,Rp\n2.240,4.7,540 sold,Kab. Purworejo,https://shopee.co.id/Bibit-Pakan-Ternak-Kalian...
3,BIBIT KALIANDRA PAKAN TERNAK KAMBING DLL UKURA...,https://down-id.img.susercontent.com/file/id-1...,Rp\n2.500,3.0,56 sold,Kab. Purworejo,https://shopee.co.id/BIBIT-KALIANDRA-PAKAN-TER...
4,Bibit Javanese Trigona/Levicep/Pot Madu Tipis/...,https://down-id.img.susercontent.com/file/id-1...,Rp\n46.999,4.9,224 sold,Kab. Probolinggo,https://shopee.co.id/Bibit-Javanese-Trigona-Le...


## Review Data

In [16]:
review_data = []  # List untuk menyimpan semua review
review_set = set()  # Set untuk mencegah duplikasi review

for idx, url in enumerate(df['link']):
    print(f"\nMemulai scraping untuk produk {idx + 1}: {url}")
    driver.get(url)
    time.sleep(5)

    halaman = 1
    review_terulang_count = 0  # Hitung jumlah review yang sudah ada

    while True:
        print(f"Scraping halaman ke-{halaman}")

        # Ambil semua review di halaman ini
        review_list = driver.find_elements(By.XPATH, "//div[contains(@class, 'shopee-product-rating__main')]")
        print(f"Jumlah review ditemukan: {len(review_list)}")

        if len(review_list) == 0:
            print("Tidak ada review ditemukan di halaman ini. Berhenti scraping produk ini.")
            break

        for i, review in enumerate(review_list, start=1):
            try:
                # Tangkap teks dan atribut
                penulis = review.find_element(By.XPATH, ".//a[contains(@class, 'shopee-product-rating__author-name')]").text if review.find_elements(By.XPATH, ".//a[contains(@class, 'shopee-product-rating__author-name')]") else "None"
                waktu = review.find_element(By.XPATH, ".//div[contains(@class, 'shopee-product-rating__time')]").text if review.find_elements(By.XPATH, ".//div[contains(@class, 'shopee-product-rating__time')]") else "None"
                pesan = review.find_element(By.XPATH, ".//div[contains(@style, 'position: relative; box-sizing: border-box; margin: 15px 0px; font-size: 14px; line-height: 20px; color: rgba(0, 0, 0, 0.87); word-break: break-word; white-space: pre-wrap;')]").text if review.find_elements(By.XPATH, ".//div[contains(@style, 'position: relative; box-sizing: border-box; margin: 15px 0px; font-size: 14px; line-height: 20px; color: rgba(0, 0, 0, 0.87); word-break: break-word; white-space: pre-wrap;')]") else "None"

                # Buat ID unik untuk mencegah duplikasi
                review_id = (penulis, waktu, pesan)

                if review_id in review_set:
                    print(f"Review ke-{i} sudah ada. Lewatkan.")
                    review_terulang_count += 1

                    # Jika sudah ada 3 review terulang, hentikan scraping untuk produk ini
                    if review_terulang_count >= 3:
                        print("Toleransi 3 review terulang tercapai. Berhenti scraping produk ini.")
                        break

                    continue  # Lewatkan review ini dan lanjut ke review berikutnya

                review_set.add(review_id)
                
                print(waktu)
                
                # Simpan data ke list
                review_data.append({
                    "produk": f"Produk {idx + 1}",
                    "penulis": penulis,
                    "waktu": waktu,
                    "pesan": pesan,
                })

            except Exception as e:
                print(f"Error memproses data ke-{i}: {e}")

        # Jika toleransi tercapai, hentikan scraping untuk produk ini
        if review_terulang_count >= 3:
            break

        # Cek apakah tombol "Next" ada dan dapat diklik
        try:
            tombol_next = driver.find_element(By.XPATH, "//button[contains(@class, 'shopee-icon-button--right')]")

            # Jika tombol tidak aktif, keluar dari loop
            if not tombol_next.is_enabled():
                print("Tombol halaman selanjutnya tidak aktif. Scraping selesai untuk produk ini.")
                break

            # Klik tombol "Next"
            tombol_next.click()
            print("Berhasil klik tombol halaman selanjutnya.")
            time.sleep(3)

            halaman += 1

        except NoSuchElementException:
            print("Tombol halaman selanjutnya tidak ditemukan. Scraping selesai untuk produk ini.")
            break
        except ElementClickInterceptedException:
            print("Tombol halaman selanjutnya tidak dapat diklik. Scraping selesai untuk produk ini.")
            break

print("Scraping selesai untuk semua produk.")



Memulai scraping untuk produk 1: https://shopee.co.id/Benih-Kaliandra-Merah-Pagoda-Pakan-Ternak-isi-10gram-Bibit-Pakan-Ternak-Super-i.441018096.14749395526?sp_atk=0cbac1e3-db67-49e5-86c6-b4e42458bfe9&xptdk=0cbac1e3-db67-49e5-86c6-b4e42458bfe9
Scraping halaman ke-1
Jumlah review ditemukan: 6
2022-02-20 09:27
2024-12-05 14:46
2022-06-11 08:03
2024-09-07 10:35
2024-02-02 08:52
2024-07-02 18:37
Berhasil klik tombol halaman selanjutnya.
Scraping halaman ke-2
Jumlah review ditemukan: 6
2025-03-18 17:07
2022-05-17 12:48
2024-10-28 09:49
2024-03-13 21:20
2024-10-17 11:27
2022-06-13 10:59
Berhasil klik tombol halaman selanjutnya.
Scraping halaman ke-3
Jumlah review ditemukan: 6
2022-06-15 09:31
2024-07-21 16:09
2024-12-03 13:07
2024-12-08 18:49
2024-08-13 17:15
2024-09-29 12:16
Berhasil klik tombol halaman selanjutnya.
Scraping halaman ke-4
Jumlah review ditemukan: 6
2024-08-18 08:56
2025-04-09 11:18
2025-02-12 10:49
2025-03-23 21:27
2024-07-01 03:04
2024-03-25 16:46
Berhasil klik tombol halam

Review ke-2 sudah ada. Lewatkan.
Review ke-3 sudah ada. Lewatkan.
Toleransi 3 review terulang tercapai. Berhenti scraping produk ini.
Scraping selesai untuk semua produk.


In [17]:
df_review = pd.DataFrame(review_data)

In [28]:
df_review

,produk,penulis,waktu,pesan
0,Produk 1,None,2022-02-20 09:27,"Pengirimannya cepat bungkus nya juga aman, isi..."
1,Produk 1,None,2024-12-05 14:46,Kualitas: bagus\nKegunaan: bagus\nUkuran: stan...
2,Produk 1,None,2022-06-11 08:03,Alhamdulillah sampai moga2 bisa tumbuh dan dap...
3,Produk 1,didikwahyu833,2024-09-07 10:35,Kualitas: bagus\nKegunaan: bibit pakan ternak\...
4,Produk 1,bulanbulan833,2024-02-02 08:52,Kualitas: baik\nCocok Untuk: belum ditanam/bar...
...,...,...,...,...
151,Produk 5,None,2024-07-04 07:34,None
152,Produk 5,8s202ufw1f,2024-07-07 20:47,None
153,Produk 5,rudiisbandi,2024-07-07 13:18,None
154,Produk 5,8s202ufw1f,2024-07-06 13:11,None


In [29]:
df.to_excel('Calliandra_products_from_shopee.xlsx', index=False)

In [30]:
df_review.to_excel('Calliandra_products_review_from_shopee.xlsx', index=False)